In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir='LLM documents').load_data()


In [40]:
documents[0].doc_id

'7f6bae63-6550-42f6-a8c2-4042ef2c39ee'

In [24]:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 340/340 [00:03<00:00, 107.13it/s]


In [25]:
query_engine = index.as_query_engine()

In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("What is a transformer")
pprint_response(response, show_source=True)

Final Response: A transformer is a sequence-to-sequence model that
consists of an encoder and a decoder. The encoder converts input text
into a representation, which is then passed to the decoder for
generating output text. The transformer architecture was developed at
Google in 2017 for tasks like translation, where it can convert
sequences from one domain into sequences in another domain.
______________________________________________________________________
Source Node 1/2
Node ID: 484185cd-7b42-469d-a346-e9f0edcc59f8
Similarity: 0.8106627408217553
Text: Foundational Large Language Models & Text Generation 9 September
2024 Transformer The transformer architecture was developed at Google
in 2017 for use in a translation model.1  It’s a sequence-to-sequence
model capable of converting sequences from one domain  into sequences
in another domain. For example, translating French sentences to
English ...
______________________________________________________________________
Source Node 2/

In [34]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
postprocess = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retriever,
                                    node_postprocessors=[postprocess])

In [35]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("What is a transformer")
pprint_response(response, show_source=True)

Final Response: A transformer is a sequence-to-sequence model
developed at Google in 2017 for tasks like translation. It consists of
an encoder and a decoder, with the encoder converting input text into
a representation that is then used by the decoder to generate output
text. The transformer architecture includes multiple layers such as
Multi-Head Attention, Add & Norm, Feed-Forward, Linear, and Softmax,
with the hidden layers being where the main processing occurs.
______________________________________________________________________
Source Node 1/1
Node ID: 484185cd-7b42-469d-a346-e9f0edcc59f8
Similarity: 0.8106367030469969
Text: Foundational Large Language Models & Text Generation 9 September
2024 Transformer The transformer architecture was developed at Google
in 2017 for use in a translation model.1  It’s a sequence-to-sequence
model capable of converting sequences from one domain  into sequences
in another domain. For example, translating French sentences to
English ...


In [38]:
import os.path
from llama_index.core import (VectorStoreIndex,
                              SimpleDirectoryReader,
                              StorageContext,
                              load_index_from_storage)

#Check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    #load documents and create index
    documents = SimpleDirectoryReader('LLM documents).load_data()
    index = VectorStoreIndex.from_documents(documents=documents)
    #store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    #load existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context=storage_context)
query_engine = index.as_query_engine()
response = query_engine.query("What is a transformer ?")
pprint_response(response=response)


Generating embeddings:  96%|█████████▌| 23/24 [1:05:13<02:50, 170.17s/it]


Final Response: A transformer is a sequence-to-sequence model that
consists of an encoder and a decoder. The encoder converts input text
into a representation, which is then passed to the decoder for
generating the output text. It was developed at Google in 2017 for
tasks like translation, where it can convert sequences from one domain
into sequences in another domain.


# Testing the app

In [42]:
import requests

url = "http://localhost:5601/query"
query = "What are the challenges in cybersecurity"

response = requests.get(url, params={"text": query})
print(response.json())


{'answer': 'Challenges in cybersecurity include horizontal and vertical scalability, availability, data consistency, real-time updates, backups, access control, compliance, and other considerations related to managing embeddings and vector stores. Additionally, managing mutations in embeddings over time, balancing the cost of frequently updating embeddings, and ensuring a well-defined automated process for storing, managing, and possibly purging embeddings from vector databases are also significant challenges in cybersecurity.'}


In [44]:
import requests

# File path to upload
file_path = 'LLM.pdf'

# Endpoint URL
url = "http://localhost:5601/uploadFile"

# Send the file
with open(file_path, 'rb') as f:
    files = {'file': f}
    response = requests.post(url, files=files)

# Print the response
print(response.json())


{'message': 'File uploaded and inserted into index!'}


In [45]:
import requests

url = "http://localhost:5601/query"
query = "What are different types of prompting techniques?"

response = requests.get(url, params={"text": query})
print(response.json())

{'answer': 'Different types of prompting techniques include prompt expansion, semantic search, business rules, and coherence checks.'}
